<a href="https://colab.research.google.com/github/pooniavaibhav/pytorchBasics/blob/main/BasicImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset,DataLoader

In [ ]:
# load the dataset using pandas-
data = pd.read_csv("/diabetes.csv")

In [ ]:
# taking x and y and converting into numpy array using .values
x = data.iloc[:,:-1].values
y_string = data.iloc[:,-1].values

In [ ]:
# convert the string into 0 and 1
y_int = [1 if i=="positive" else 0 for i in y_string]

In [ ]:
# convert it to numpy array-
y = np.array(y_int, dtype='float64')

In [ ]:
#Normalisation-
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
# convert the arrays to pytorch tensors
x = torch.tensor(x)
y = torch.tensor(y)

In [ ]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [ ]:
#since we will be using binary cross entropy we need to convert y into 2D-
y = torch.tensor(y).unsqueeze(1)
y.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([768, 1])

In [ ]:
# build custom pytorch datatset-
class Dataset(Dataset):

  def __init__(self,x,y):
    self.x = x
    self.y = y

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return len(self.x)

In [ ]:
dataset = Dataset(x,y)

In [ ]:
#load the  data to your dataloader for batch processing and shuffling-
train_loader = DataLoader(dataset = dataset, batch_size=32, shuffle = True)

In [ ]:
#lets have a look at the data loader-
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
  print("For one iteration (batch), there is:")
  print("Data:     {}".format(x.shape))
  print("Labels:    {}".format(y.shape))
  break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:     torch.Size([32, 7])
Labels:    torch.Size([32, 1])


In [ ]:
#building a neural network-
class Model(nn.Module):
  def __init__(self,input_features,output_features):
    super(Model,self).__init__()
    self.fc1 = nn.Linear(input_features, 5)
    self.fc2 = nn.Linear(5,4)
    self.fc3 = nn.Linear(4,3)
    self.fc4 = nn.Linear(3,output_features)
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()


  def forward(self,x):
    out = self.fc1(x)
    out = self.tanh(out)
    out = self.fc2(out)
    out = self.tanh(out)
    out = self.fc3(out)
    out = self.tanh(out)
    out = self.fc4(out)
    out = self.sigmoid(out)
    return out

In [ ]:
#create the network (an object of the Net class)
net = Model(7,1)
# In cross entropy the input and the output should have the same shape.
# size_average = True --> the losses are averaged over observations for each minibatch
criterion  = torch.nn.BCELoss(size_average=True)
#we will use SGD with momentum with a learning rate of 0.1
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# training the network-
epochs = 200
for epoch in range(200):
  for inputs,labels in train_loader:
    inputs = inputs.float()
    labels = labels.float()
    # forward prop
    outputs = net(inputs)
    # loss clculation
    loss = criterion(outputs, labels)
    #clear the gradient buffer
    optimizer.zero_grad()
    # Backprop
    loss.backward()
    #update weight-
    optimizer.step()
  #Accuracy calculation
  output = (outputs>0.5).float()
  accuracy = (outputs == labels).float().mean()
  #print stats
  print("Epochs {}/{}, Loss: {:,.3f}, Accuracy: {:,.3f}".format(epoch+1, epochs, loss, accuracy))


Epochs 1/200, Loss: 0.443, Accuracy: 0.000
Epochs 2/200, Loss: 0.638, Accuracy: 0.000
Epochs 3/200, Loss: 0.427, Accuracy: 0.000
Epochs 4/200, Loss: 0.471, Accuracy: 0.000
Epochs 5/200, Loss: 0.450, Accuracy: 0.000
Epochs 6/200, Loss: 0.282, Accuracy: 0.000
Epochs 7/200, Loss: 0.419, Accuracy: 0.000
Epochs 8/200, Loss: 0.635, Accuracy: 0.000
Epochs 9/200, Loss: 0.370, Accuracy: 0.000
Epochs 10/200, Loss: 0.522, Accuracy: 0.000
Epochs 11/200, Loss: 0.373, Accuracy: 0.000
Epochs 12/200, Loss: 0.548, Accuracy: 0.000
Epochs 13/200, Loss: 0.378, Accuracy: 0.000
Epochs 14/200, Loss: 0.388, Accuracy: 0.000
Epochs 15/200, Loss: 0.482, Accuracy: 0.000
Epochs 16/200, Loss: 0.455, Accuracy: 0.000
Epochs 17/200, Loss: 0.565, Accuracy: 0.000
Epochs 18/200, Loss: 0.509, Accuracy: 0.000
Epochs 19/200, Loss: 0.546, Accuracy: 0.000
Epochs 20/200, Loss: 0.654, Accuracy: 0.000
Epochs 21/200, Loss: 0.538, Accuracy: 0.000
Epochs 22/200, Loss: 0.542, Accuracy: 0.000
Epochs 23/200, Loss: 0.356, Accuracy: 0.0